# deviceConnect Appium iOS Python Demonstration

This Jupyter notebook contains Python code that is useful for reproducing and diagnosing Appium issues on iOS.

It is designed to be useful as boilerplate for an issue-specific notebook. Just duplicate it, update the variables in the **Configuration** section, run all the cells to start and connect to an Appium session, then start addding to the **Testing** section at the bottom.

## Prerequisities

- A deviceConnect server with attached iOS device(s)
- A user account on the deviceConnect server, and a user-specific API key
- [Jupyter](http://jupyter.org)
   - On macOS, the easiest way to install Jupyter is to first install [Homebrew](http://brew.sh), then enter the command `brew install jupyter`.
   - When Jupyter is installed, open a command line prompt, go to the directory where this notebook is located, and run the command `jupyter notebook` to start a Jupyter server.

## Libraries

First, import often-useful standard libraries.

In [ ]:
from __future__ import print_function

import logging
import os
import sys

Import the Appium and Selenium client libraries.

If these libraries are not available, try running this command line:

    pip install Appium-Python-Client

If you don't have "pip", try

    sudo easy_install pip
    
If pip fails due to permissions issues, try this:

    pip install Appium-Python-Client --user
    
See the [appium/python-client](https://github.com/appium/python-client) repository for more information.

Also see the [Selenium Python API](https://seleniumhq.github.io/selenium/docs/api/py/api.html 

In [ ]:
from appium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Configuration

Define some variables for our capabilities and connection parameters.

In [ ]:
DC_APPIUM_HOST_URL = 'http://10.211.55.33'
DC_APPIUM_USER_NAME = 'kris.johnson@mobilelabsinc.com'
DC_APPIUM_API_KEY = '5a28bf2d-c494-4ee5-8881-89a2721e12aa'
DC_APPIUM_DEVICE_ID = '0761b3b4-a5c9-443b-b2be-3810e8251d90'
DC_APPIUM_BUNDLE_ID = 'com.mobilelabsinc.TrustAutoTest-Touchpose'

Configure the logging module so that we can see debug output. (These will be printed on the Jupyter server's console, not here in this notebook.)

In [ ]:
%config Application.log_level="DEBUG"

## Utility Functions

The following cells contain functions that are often useful 

In [ ]:
def capabilities():
    """Return Appium capabilities for this test.
    """
    caps = {
        'deviceConnectUserName'      : DC_APPIUM_USER_NAME,
        'deviceConnectApiKey'        : DC_APPIUM_API_KEY,
        'deviceConnectDevice'        : DC_APPIUM_DEVICE_ID,
        'deviceConnectIgnoreSession' : 'true',
        'newCommandTimeout'          : 1000,
        'platformName'               : 'iOS',
        'automationName'             : 'xcuitest',
        'bundleId'                   : DC_APPIUM_BUNDLE_ID,
    }

    return caps

def connect_appium(capabilities):
    """Connect to the server and launch the app under Appium control.

    Returns a webdriver object.
    """
    url = '{0}/Appium'.format(DC_APPIUM_HOST_URL)

    logging.info('connecting to device {0} on {1}...'
        .format(DC_DEVICE_ID, url))
    driver = webdriver.Remote(url, capabilities)
    logging.info('connected')

    return driver

def click_button_with_label(driver, label):
    """Click the button with the specified label value.
    """
    xpath = ('//XCUIElementTypeButton[@label="{0}"]'
        .format(escape_double_quotes(label)))
    click_element_with_xpath(driver, xpath)

def click_element_with_xpath(driver, xpath):
    """Wait for element to appear, then click it.
    """
    wait_for_element_with_xpath(driver, xpath)
    driver.find_element_by_xpath(xpath).click()

def set_text_of_element_with_xpath(driver, xpath, newText):
    """Wait for element to appear, then set its text.
    """
    wait_for_element_with_xpath(driver, xpath)
    element = driver.find_element_by_xpath(xpath)

    # .set_text() often doesn't work for iOS, so we use .set_value().
    # Also, call .clear() first or we may just append to
    # existing value.

    element.click()
    element.clear()
    element.set_value(newText)

def text_of_element_with_xpath(driver, xpath):
    """Wait for element to appear, then get its text.
    """
    element = wait_for_element_with_xpath(driver, xpath)
    return element.text

def wait_for_element_with_xpath(driver, xpath, timeoutSeconds=15):
    """Wait until an element with the specified XPath appears.
    """
    logging.info('looking for element with XPath {0}...'
        .format(xpath))

    WebDriverWait(driver, timeoutSeconds).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

    logging.info('found {0}'.format(xpath))

def wait_for_element_with_xpath_to_contain_text(driver, xpath, text, timeoutSeconds=15):
    """Wait until an element with the specified XPath contains the specified text.

    Returns reference to the found element.

    Throws TimeoutException if the condition never occurs.
    """
    logging.info('waiting for text \"%s\" in element with XPath %s...',
        text, xpath)

    element = WebDriverWait(driver, timeoutSeconds).until(
        EC.text_to_be_present_in_element((By.XPATH, xpath), text))

    logging.info('found \"%s\" in element with XPath %s',
        text, xpath)

    return element

def xpath_for_allow_button():
    """Return Xpath for the OK/Allow/Always Allow button in a system permissions alert.
    """
    return '//XCUIElementTypeButton[@label="Allow" or @label="Always Allow" or @label="OK"]'

def wait_for_allow_button(driver, timeoutSeconds=15):
    """Wait until a popup with an OK/Allow/Always Allow button appears.

    Returns reference to the element.

    Throws exception if no such button appears.
    """
    return wait_for_element_with_xpath(
        driver, xpath_for_allow_button(), timeoutSeconds)

def click_allow_button(driver):
    """Click 'OK/Allow/Always Allow' button.
    """
    return click_element_with_xpath(driver, xpath_for_allow_button())

def escape_double_quotes(s):
    """If string contains any double-quote (") characters, replace with '\"'.

    Returns escaped string.
    """
    return s.replace('"', '\\"')

def save_page_source_to_file(driver, filename):
    """Saves current page source to specified file.
    """
    f = open(filename, 'w')
    f.write(driver.page_source)
    f.close()

## Connecting

The following cells will, when executed, create an Appium session on the device and connect to it.

Verify that the correct capability values are being used:

In [ ]:
caps = capabilities()
caps

Using those capabilities, connect to the deviceConnect Appium server:

In [ ]:
driver = connect_appium(caps)

Now that we are connected, retrieve the page source to verify that Appium is working and we see what we expect to see:

In [ ]:
driver.page_source

## Testing

Now that we are connected, we can experiment.  Enter commands and expressions in the cells below.

In addition to the utility functions above, these commands and expressions are often useful:

    driver.save_screenshot(filename)
    driver.screenshot_as_png
    driver.quit()
    driver.find_element_by_xpath(xpath)   | driver.find_elements_by_xpath(xpath)
    driver.find_element_by_name(name)     | driver.find_elements_by_xpath(name)
    driver.find_element_by_tag_name(name) | driver.find_elements_by_tag_name(name)
    
    element.rect
    element.size
    element.text
    element.is_displayed()
    element.tag_name
    element.id
    element.is_enabled()
    element.is_selected()
    element.get_attribute('name')
    element.get_attribute('label')
    element.get_attribute('type')
    element.get_attribute('visible')
    element.get_attribute('value')
    element.get_attribute('accessible')
    element.get_attribute('UID')
    element.click()
    element.clear()
    element.send_keys(text)

API docs:

   - [Selenium WebDriver](https://seleniumhq.github.io/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webdriver.html)
   - [Selenium WebElement](https://seleniumhq.github.io/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webelement.html)
   - [Appium Python Client](https://github.com/appium/python-client)